<a href="https://colab.research.google.com/github/thiagoribeiro00/neuroscience-computational/blob/main/snn_leaky_integrate_and_fire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Execute esta célula no Google Colab para instalar Norse
!pip install norse torch torchvision --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s e

Rede Neural com Neurônio LIF


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from norse.torch import LIFParameters, LIFState, LIFCell

# Define o dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Baixa o dataset MNIST (dígitos manuscritos)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x * 32)  # aumenta a intensidade do input
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)


Definindo a Rede com LIFCell


In [ ]:
class SNN_LIF_Model(nn.Module):
    def __init__(self):
        super().__init__()

        # Camada densa de entrada
        self.fc1 = nn.Linear(28*28, 128)

        # Neurônio LIF (Leaky Integrate-and-Fire)
        self.lif1 = LIFCell()

        # Camada de saída
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        seq_length = 10  # Número de steps no tempo
        batch_size = x.shape[0]

        # Inicializa o estado do neurônio LIF
        lif_state = None

        # Armazena os spikes ao longo do tempo
        outputs = torch.zeros(batch_size, 10, device=x.device)

        # Entrada deve ser achatada para vetor (imagem 28x28 → vetor de 784)
        x = x.view(batch_size, -1)

        for t in range(seq_length):
            z = self.fc1(x)

            # Passa pelo neurônio LIF (retorna spike e novo estado)
            s, lif_state = self.lif1(z, lif_state)

            # Passa pela camada de saída
            out = self.fc2(s)

            # Soma as saídas ao longo do tempo
            outputs += out

        return outputs / seq_length  # Média das ativações


Treinando o Modelo

In [ ]:
model = SNN_LIF_Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

epochs = 5  # Pode aumentar para 5 ou 10 se quiser

for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss/len(trainloader):.4f}")


Epoch 1 - Loss: 0.3055
Epoch 2 - Loss: 0.1679
Epoch 3 - Loss: 0.1344
Epoch 4 - Loss: 0.1144
Epoch 5 - Loss: 0.1029


In [ ]:
# Avaliação da acurácia no conjunto de treino
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Acurácia no conjunto de treino: {100 * correct / total:.2f}%")


Acurácia no conjunto de treino: 97.47%


## 🧠 Spiking Neural Network com Neurônios LIF — Explicação

### 📌 Pré-processamento de dados (MNIST)
Utilizamos o dataset **MNIST** (dígitos manuscritos), comum em tarefas de classificação.

As imagens foram **normalizadas e multiplicadas por 32** para simular correntes de entrada mais fortes aos neurônios.

---

### 🏗️ Construção do Modelo SNN com LIF

Criamos uma rede neural com a seguinte estrutura:

- 🔹 **Camada densa (Linear)** que transforma a imagem de entrada (28x28 = 784 pixels) em um vetor de 128 neurônios.
- 🔹 **Neurônio LIF (`LIFCell`)**, que recebe essa entrada e gera *spikes* ao longo do tempo.
- 🔹 **Camada de saída (Linear)** que converte os spikes acumulados em uma predição de classe (0 a 9).

A propagação é feita ao longo de **várias janelas temporais** (*time steps*), simulando a dinâmica de um neurônio biológico.

---

### 🏋️ Treinamento da Rede

- ✅ Utilizamos `CrossEntropyLoss` para calcular o erro de classificação.
- ✅ O otimizador **Adam** ajusta os pesos da rede com base nos erros.
- ✅ A rede é treinada por **1 época** (mas pode ser ajustado para mais).
- ✅ O output da rede é a **média das ativações temporais**, simulando uma **taxa de disparo (firing rate)**.

---

### 🔍 Conceitos Importantes Demonstrados

| Conceito                | Explicação |
|-------------------------|------------|
| **LIF Neuron**          | Modelo que acumula corrente e dispara um *spike* quando o limiar é atingido. Após o disparo, o potencial de membrana é resetado. |
| **Spikes ao longo do tempo** | A rede simula como os neurônios se comportam em diferentes instantes, processando entradas dinâmicas. |
| **SNN vs ANN**          | Em vez de usar valores contínuos (ex: ReLU), usamos *spikes* binários (0 ou 1) ao longo do tempo. |
| **Treinamento supervisionado** | Apesar da natureza esparsa dos *spikes*, usamos funções de perda tradicionais (como `CrossEntropy`) e **backpropagation**. |

---

### 🧪 Aplicações Práticas da Arquitetura

- 🔸 Reconhecimento de padrões em tempo real  
- 🔸 Dispositivos embarcados (baixo consumo de energia)  
- 🔸 Simulações neuromórficas  
- 🔸 Robótica biológica

